<a href="https://colab.research.google.com/github/saigirishgilly98/Major_Project/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install scrapy

     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 2.6MB 9.7MB/s 
     |████████████████████████████████| 3.2MB 26.3MB/s 
     |████████████████████████████████| 245kB 34.2MB/s 
     |████████████████████████████████| 3.1MB 44.0MB/s 
     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=e0efacc386fef950bbffb727e3cb79f73f6f9a3bc63c3de7bd09c1e8cc62a7f1
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11517 sha256=a70532fc9d0dd07a9ef1d53bf1718b046fa79ecdc4f13b7a604466bb0d2c069c
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyD

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
class DC_Chapter_Spider(scrapy.Spider):
  
  name = "dc_chapter_spider"

  def start_requests(self):
    url = 'https://www.datacamp.com/courses/all'
    yield scrapy.Request(url = url, callback = self.parse_front)

  def parse_front(self, response):
    # Narrow in on the course blocks
    course_blocks = response.css('div.course-block')

    # Direct to the course links
    course_links = course_blocks.xpath('./a/@href')

    # Extract the links (as a list of strings)
    links_to_follow = course_links.extract()

    # Follow the links to the next parser
    for url in links_to_follow:
      yield response.follow(url = url, callback = self.parse_pages)

  def parse_pages(self, response):
    
    # Direct to the course title text
    crs_title = response.xpath('//h1[contains(@class,"title")]/text()')
    
    # Extract and clean the course title text
    crs_title_ext = crs_title.extract_first().strip()
    
    # Direct to the chapter titles text
    ch_titles = response.css( 'h4.chapter__title::text' )
    
    # Extract and clean the chapter titles text
    ch_titles_ext = [t.strip() for t in ch_titles.extract()]
    
    # Store this in our dictionary
    dc_dict[ crs_title_ext ] = ch_titles_ext


dc_dict = dict()

process = CrawlerProcess()
process.crawl(DC_Chapter_Spider)
process.start()

In [ ]:
dc_dict

In [ ]:
import pandas as pd
import csv

In [ ]:
class Payscale_Spider(scrapy.Spider):

  name = "payscale_spider"

  def start_requests(self):
    url = 'https://www.payscale.com/research/IN/Job/Information-Technology'
    yield scrapy.Request(url = url, callback = self.parse_jobs)

  def parse_jobs(self, response):

    job_blocks = response.css('div.subcats__links')
    job_links = job_blocks.xpath('./a/@href').extract()
    for link in job_links:
      yield response.follow(url = 'https://www.payscale.com' + link, callback = self.parse_salary)

  def parse_salary(self, response):

    # Extract the job
    job_title = response.url[41:len(response.url)-7]

    # Extract the average salary
    avg_salary = response.css('div.spotlight__value::text').extract_first()

    # Extract bonus, commission and profit sharing
    bonus_commision_profitsharing_value = response.css('div.spotlight__stat-value::text').extract()
    bonus_commision_profitsharing_label = response.css('div.spotlight__stat-label::text').extract()

    # Extract percentile of salary
    percentile_salary_ = response.css('div.percentile-chart__label')
    percentile_salary_name = percentile_salary_.xpath('./div[1]/text()').extract()
    percentile_salary_value = percentile_salary_.xpath('./div[2]/text()').extract()

    # Extract salary_distribution, company salaries and related job salaries
    salary_distribution_title = response.css('div.tablerow__title > a::text').extract()
    salary_distribution_value = response.css('div.tablerow__value::text').extract()

    # Extract career paths
    career_paths = response.xpath('//div[@class = "tree-chart__leaf_container"]/div[1]/text()').extract()

    # Extract popular skill and salaries
    skills_name = response.css('a.popular-skill__name::text').extract()
    skills_salaries = response.css('div.popular-skill__pay::text').extract()
    
    # Extract pay by experience
    pay_by_experience = response.css('div.job__blurb::text').extract()

    # Extract pay difference by location, experience affects salary, skills affect salary
    pay_by_location_name = response.css('div.entry > div.name > a::text').extract()
    pay_by_location_symbol = response.css('div.entry > div.arrow > span::attr(class)').extract()
    pay_by_location_percent = response.css('div.entry > div.arrow::text').extract()

    # Extract years of experience, gender breakdown
    experience_name = response.css('div.pie-chart__name::text').extract()
    experience_percent = response.css('div.pie-chart__value::text').extract() 

    # Extract health benefits
    health_benefits_name = response.css('div.health-benefits__item-name::text').extract()
    health_benefits_value = response.css('div.health-benefits__item-value::text').extract()

    # Updating all the values in the dictionary
    salary_dict['average_salary'] = avg_salary
    for label, value in zip(bonus_commision_profitsharing_label, bonus_commision_profitsharing_value):
      salary_dict[label] = value
    for label, value in zip(percentile_salary_name, percentile_salary_value):
      salary_dict[label] = value
    for label, value in zip(salary_distribution_title, salary_distribution_value[5:]):
      salary_dict[label] = value 
    salary_dict['career_paths'] = career_paths
    for label, value in zip(skills_name, skills_salaries):
      salary_dict[label + '_salary'] = value
    pay_by_experience_salary = list()
    for word in pay_by_experience[2].split():
      if word.startswith('₹'):
        pay_by_experience_salary.append(word)
    salary_dict['1yr'] = pay_by_experience_salary[0]
    salary_dict['1-4yrs'] = pay_by_experience_salary[1]
    salary_dict['5-9yrs'] = pay_by_experience_salary[2]
    salary_dict['10-19yrs'] = pay_by_experience_salary[3]
    salary_dict['20+yrs'] = pay_by_experience_salary[4]
    for label, symbol, value in zip(pay_by_location_name, pay_by_location_symbol, pay_by_location_percent):
      if symbol == 'negative':
        salary_dict[label + '_percent'] = '-' + value
      else:
        salary_dict[label + '_percent'] = value
    for label, value in zip(experience_name, experience_percent[::2]):
      salary_dict[label] = value
    for label, value in zip(health_benefits_name, health_benefits_value[::2]):
      salary_dict[label] = value
    salary_dict['Job Title'] = job_title

    # Storing the values in csv file
    df = pd.DataFrame(salary_dict)
    df.to_csv('' + job_title + '.csv')

salary_dict = dict()

process = CrawlerProcess()
process.crawl(Payscale_Spider)
process.start()

2020-10-16 15:09:57 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: scrapybot)
2020-10-16 15:09:57 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jul 17 2020, 12:50:27) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2020-10-16 15:09:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-10-16 15:09:57 [scrapy.crawler] INFO: Overridden settings:
{}
2020-10-16 15:09:57 [scrapy.extensions.telnet] INFO: Telnet Password: 694b012c46cfb37e
2020-10-16 15:09:57 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-10-16 15:09:57 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloa

In [ ]:
salary_dict

{'.NET Software Developer / Programmer': '₹151k - ₹968k',
 '.NET_percent': '-8%',
 '.NET_salary': '₹803,979',
 '1-4yrs': '₹501,168',
 '10%': '₹261k',
 '10-19yrs': '₹1,507,035',
 '1yr': '₹394,619',
 '20+yrs': '₹813,892.',
 '3i Infotech Inc.': '₹158k',
 '5-9yrs': '₹886,543',
 '90%': '₹1m',
 'ABAP/4_salary': '₹511,531',
 'ASP.NET Framework_percent': '-11%',
 'ASP.NET Framework_salary': '₹412,972',
 'Accentcare, Inc.': '₹297k - ₹2m',
 'Accenture': '₹447k',
 'Accenture Technology Solutions': '₹455k',
 'Acceptance Testing_percent': '7%',
 'Accountant': '₹151k - ₹980k',
 'Adobe Photoshop_salary': '₹359,159',
 'Advanced Business Application Programming (ABAP) Language_salary': '₹485,059',
 'Agile Software Development_percent': '6%',
 'Agile Software Development_salary': '₹2,057,899',
 'Ahmedabad, Gujarat_percent': '-22%',
 'Algorithm Development_salary': '₹1,391,324',
 'Amadeus India Pvt Ltd': '₹2m',
 'Amazon Web Services (AWS)_percent': '-9%',
 'Amazon Web Services (AWS)_salary': '₹483,333',


In [ ]:
df2 = pd.read_csv('pay_scale.csv')
df2.head()

ParserError: ignored